## Instalação e configuração

É necessário a instalação de algumas bibliotecas essenciais para o desenvolvimento de modelos de aprendizado de máquina e processamento de linguagem natural (NLP):
- **Transformers, da Hugging Face:** Oferece uma vasta gama de modelos pré-treinados como BERT, GPT e T5 para tarefas de NLP.
- **Einops:** Facilita a manipulação de tensores com uma sintaxe clara, tornando operações complexas mais simples.
- **Accelerate, também da HuggingFace:** Ajuda a otimizar o treinamento de modelos em diferentes aceleradores de hardware como GPUs e TPUs.
- **BitsAndBytes:** Possibilita a quantização eficiente de modelos grandes, reduzindo o consumo de memória em PyTorch.

In [1]:
# do professor:
#!pip install -q transformers==4.48.2 einops accelerate bitsandbytes

# Foi necessário instalar cada lib separadamente no ambiente linux com conda
# !pip install transformers
# !pip install einops
# !pip install accelerate
# !pip install bitsandbytes

- AutoModelForCausalLM: Uma classe que fornece um modelo de linguagem causal (ou autoregressivo) pré-treinado (por exemplo, GPT-2, GPT-3) que **são adequados para tarefas de geração de texto**.
- AutoTokenizer: Uma classe que fornece um tokenizador que corresponde ao modelo. O tokenizador é responsável por converter texto em tokens numéricos que o modelo pode entender.
- pipeline: fornece uma interface simples e unificada para várias tarefas de PLN, facilitando a execução de tarefas como geração de texto, classificação e tradução.
- BitsAndBytesConfig: Uma classe para configuração de quantização e outras otimizações de baixo nível para melhorar a eficiência computacional.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

/home/wolf/anaconda3/envs/spero-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-30 16:12:54.860365: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-30 16:12:54.884162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756581174.912734   55537 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756581174.921114   55537 cuda_blas.cc:

In [8]:
import torch
import getpass
import os
from dotenv import load_dotenv

# Quando usa-se a palavra "cuda" estamos trabalhando com GPU
device = "cuda:0" if  torch.cuda.is_available() else "cpu"
device

'cuda:0'

SEED para garantir a reprodutibilidade entre diferentes experimentos e execuções.

Ou seja, sempre que o código for executado sempre serão gerados os mesmos resultados.

In [6]:
torch.random.manual_seed(42)

### Definição do Token da Hugging Face

In [ ]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

# O getpass também é interessante pois solicita ao usuário digitar uma "senha"
# getpass.getpass()

### Carregando o modelo

Iremos utilizar o modelo Phi-3-mini-4k-instruct:

Foi escolhido pois é open source, acessível e consegue responder bem em português (embora ainda seja melhor em inglês). Verá que muitos modelos não entendem esse idioma, e os que entendem são muito pesados para executarmos em nosso ambiente, ou seja, precisamos acessar via alguma API ou interface web, como o ChatGPT. Porém, nesse momento queremos explorar soluções open source, para obtermos maior liberdade.

[Acessar modelo Phi-3-mini-4k-instruct no Hugging Face](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)

Este modelo tem 4k de tamanho, isso se refere ao comprimento da sequência, que neste caso é de 4000 tokens. Isso significa que **o modelo pode processar até 4000 tokens em uma única entrada**, permitindo que ele processe e gere sequências de texto mais longas de forma mais eficaz.



In [12]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    id_model, # Modelo que será baixado
    device = "cuda", # Deve ser carregada em uma GPU habilitada para CUDA
    torch_dtype = "auto",
    trust_remote_code = True, # Permite o carregamento de código personalizado
    attn_implementation="eager" # Método de implementação para o mecanismo de atenção
)

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 2 files:   0%|          | 0/2 [00:09<?, ?it/s]


KeyboardInterrupt: 